# MongoDB Operations - Retail Pricing Data

This notebook demonstrates MongoDB CRUD operations and aggregation pipelines for retail pricing data.

## Topics Covered:
1. Connecting to MongoDB Atlas
2. CRUD Operations (Create, Read, Update, Delete)
3. Aggregation Pipeline Examples
4. Comparison with SQL Queries ( As asked in requirments)


In [50]:
# Import required libraries
import os
import sys
from pathlib import Path
import json
from pprint import pprint

# Add parent directory to path to import modules
sys.path.append(str(Path().resolve().parent))

from mongodb_operations import MongoDBOperations
from data_transformer import transform_csv_to_documents

print("Libraries imported successfully!")


Libraries imported successfully!


## 1. Connect to MongoDB


In [ ]:
mongo = MongoDBOperations()

print("Connection successful!")
print(f"Database: {mongo.database_name}")
print(f"Collection: {mongo.collection.name}")


Successfully connected to MongoDB!
Connection successful!
Database: retail_db
Collection: products


## 2. CREATE Operations


In [ ]:
sample_document = {
    "sku": "TEST_001_S",
    "style_id": "TEST_001",
    "product": {
        "catalog": "Test Catalog",
        "category": "Kurta",
        "weight": 0.3,
        "size": "S"
    },
    "pricing": {
        "tp": 500,
        "mrp_old": 2000,
        "final_mrp_old": 2100
    },
    "ecommerce_platforms": {
        "ajio": 2100,
        "amazon": 2100,
        "flipkart": 2100
    },
    "metadata": {
        "cheapest_platform": "ajio",
        "cheapest_price": 2100,
        "price_range": 0,
        "min_price": 2100,
        "max_price": 2100,
        "avg_price": 2100,
        "created_at": "2024-01-01T00:00:00",
        "data_source": "Test"
    }
}

doc_id = mongo.insert_one(sample_document)
print(f"Inserted document ID: {doc_id}")


Inserted document with ID: 695abe4b39988a9329cce3b3
Inserted document ID: 695abe4b39988a9329cce3b3


In [ ]:

csv_path = '../data/May-2022.csv'

print("Loading data from CSV...")
doc_ids = mongo.load_from_csv(str(csv_path), limit=100)
print(f"Inserted {len(doc_ids)} documents")


Loading data from CSV...
Loading CSV from ../data/May-2022.csv...
Limiting to 100 records
Transforming 100 records to MongoDB documents...
Successfully transformed 100 documents
Inserted 100 documents
Inserted 100 documents


## 3. READ Operations


In [ ]:
print("=== All Documents (first 5) ===")
all_docs = mongo.find_all(limit=5)
for doc in all_docs:
    print(f"SKU: {doc.get('sku')}, Category: {doc.get('product', {}).get('category')}")
    print(f"Cheapest Platform: {doc.get('metadata', {}).get('cheapest_platform')}")
    print("-" * 50)


=== All Documents (first 5) ===
SKU: Os206_3141_S, Category: Kurta
Cheapest Platform: ajio
--------------------------------------------------
SKU: Os206_3141_M, Category: Kurta
Cheapest Platform: ajio
--------------------------------------------------
SKU: Os206_3141_L, Category: Kurta
Cheapest Platform: ajio
--------------------------------------------------
SKU: Os206_3141_XL, Category: Kurta
Cheapest Platform: ajio
--------------------------------------------------
SKU: Os206_3141_2XL, Category: Kurta
Cheapest Platform: ajio
--------------------------------------------------


In [ ]:
print("=== Find by SKU ===")
sku = "Os206_3141_S"  # Replace with actual SKU from your data
doc = mongo.find_by_sku(sku)
if doc:
    print(f"Found document for SKU: {sku}")
    pprint(doc)
else:
    print(f"No document found for SKU: {sku}")


=== Find by SKU ===


Found document for SKU: Os206_3141_S
{'_id': ObjectId('695abc1b39988a9329cce284'),
 'ecommerce_platforms': {'ajio': 2200.0,
                         'amazon': 2295.0,
                         'amazon_fba': 2295.0,
                         'flipkart': 2295.0,
                         'limeroad': 2295.0,
                         'myntra': 2295.0,
                         'paytm': 2295.0,
                         'snapdeal': 2295.0},
 'metadata': {'avg_price': 2283.125,
              'cheapest_platform': 'ajio',
              'cheapest_price': 2200.0,
              'created_at': '2026-01-05T00:44:35.604532',
              'data_source': 'May-2022',
              'max_price': 2295.0,
              'min_price': 2200.0,
              'price_range': 95.0},
 'pricing': {'final_mrp_old': 2295.0, 'mrp_old': 2178.0, 'tp': 538.0},
 'product': {'catalog': 'Moments',
             'category': 'Kurta',
             'size': None,
             'weight': 0.3},
 'sku': 'Os206_3141_S',
 'style_id': 'Os206_

In [56]:
# Find by category
print("=== Find by Category (Kurta) ===")
kurtas = mongo.find_by_category("Kurta", limit=5)
print(f"Found {len(kurtas)} products in Kurta category")
for product in kurtas:
    print(f"SKU: {product.get('sku')}, Price: ₹{product.get('metadata', {}).get('cheapest_price')}")


=== Find by Category (Kurta) ===
Found 5 products in Kurta category
SKU: Os206_3141_S, Price: ₹2200.0
SKU: Os206_3141_M, Price: ₹2295.0
SKU: Os206_3141_L, Price: ₹2295.0
SKU: Os206_3141_XL, Price: ₹2295.0
SKU: Os206_3141_2XL, Price: ₹2295.0


In [57]:
# Find by price range
print("=== Find by Price Range (₹1900 - ₹2000) ===")
products = mongo.find_by_price_range(1900, 2000, limit=5)
print(f"Found {len(products)} products in price range")
for product in products:
    print(f"SKU: {product.get('sku')}, Price: ₹{product.get('metadata', {}).get('cheapest_price')}")


=== Find by Price Range (₹1900 - ₹2000) ===
Found 0 products in price range


In [58]:
# Find cheapest products
print("=== Cheapest Products (Top 10) ===")
cheapest = mongo.find_cheapest_products(limit=10)
for product in cheapest:
    print(f"SKU: {product.get('sku')}, "
          f"Category: {product.get('product', {}).get('category')}, "
          f"Price: ₹{product.get('metadata', {}).get('cheapest_price')}, "
          f"Platform: {product.get('metadata', {}).get('cheapest_platform')}")


=== Cheapest Products (Top 10) ===
SKU: Os181_5051_XL, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os181_5051_L, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os191_5052_S, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os181_5051_2XL, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os181_5051_S, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os181_5051_3XL, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os191_5052_XL, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os191_5052_L, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os191_5052_M, Category: Kurta, Price: ₹1895.0, Platform: ajio
SKU: Os181_5051_M, Category: Kurta, Price: ₹1895.0, Platform: ajio


## 4. UPDATE Operations


In [ ]:

sku = "Os214_3144"
platform = "ajio"
new_price = 2200.0

success = mongo.update_price(sku, platform, new_price)
if success:
    print(f"Price updated successfully for {sku} on {platform}")
else:
    print("Update failed")

print("Update operations ready")


No document found with SKU Os214_3144
Update failed
Update operations ready


## 5. DELETE Operations


In [ ]:

sku = "TEST_001_S"
success = mongo.delete_by_sku(sku)
if success:
    print(f"Document with SKU {sku} deleted")
else:
    print("Delete failed - document not found")

print("Delete operations ready")


Deleted document with SKU TEST_001_S
Document with SKU TEST_001_S deleted
Delete operations ready


## 6. Aggregation Pipeline Examples

### 6.1 Group by Category


In [61]:
# MongoDB Aggregation: Group by Category
print("=== Aggregation: Group by Category ===")
category_stats = mongo.aggregate_by_category()

print("\nCategory Statistics:")
for stat in category_stats:
    print(f"\nCategory: {stat.get('category')}")
    print(f"  Total Products: {stat.get('total_products')}")
    print(f"  Unique SKUs: {stat.get('unique_skus')}")
    print(f"  Average Price: ₹{stat.get('avg_price')}")
    print(f"  Price Range: ₹{stat.get('min_price')} - ₹{stat.get('max_price')}")

# SQL Equivalent:
# SELECT 
#     category,
#     COUNT(*) as total_products,
#     COUNT(DISTINCT sku) as unique_skus,
#     AVG(avg_price) as avg_price,
#     MIN(min_price) as min_price,
#     MAX(max_price) as max_price
# FROM products
# GROUP BY category
# ORDER BY total_products DESC;


=== Aggregation: Group by Category ===

Category Statistics:

Category: Kurta
  Total Products: 400
  Unique SKUs: 100
  Average Price: ₹2086.97
  Price Range: ₹1895.0 - ₹2295.0


### 6.2 Platform Price Analysis


In [62]:
print("=== Aggregation: Platform Price Analysis ===")
platform_stats = mongo.aggregate_by_platform()

print("\nPlatform Statistics:")
for stat in platform_stats:
    print(f"\nPlatform: {stat.get('platform')}")
    print(f"  Products Available: {stat.get('products_count')}")
    print(f"  Average Price: ₹{stat.get('avg_price')}")
    print(f"  Price Range: ₹{stat.get('min_price')} - ₹{stat.get('max_price')}")

# Note: This is more complex in SQL due to denormalized platform data
# Would require UNPIVOT or multiple queries


=== Aggregation: Platform Price Analysis ===

Platform Statistics:

Platform: {'ajio': 2195.0, 'amazon': 2195.0, 'amazon_fba': 2195.0, 'flipkart': 2195.0, 'limeroad': 2195.0, 'myntra': 2195.0, 'paytm': 2195.0, 'snapdeal': 2195.0}
  Products Available: 64
  Average Price: ₹None
  Price Range: ₹{'ajio': 2195.0, 'amazon': 2195.0, 'amazon_fba': 2195.0, 'flipkart': 2195.0, 'limeroad': 2195.0, 'myntra': 2195.0, 'paytm': 2195.0, 'snapdeal': 2195.0} - ₹{'ajio': 2195.0, 'amazon': 2195.0, 'amazon_fba': 2195.0, 'flipkart': 2195.0, 'limeroad': 2195.0, 'myntra': 2195.0, 'paytm': 2195.0, 'snapdeal': 2195.0}

Platform: {'ajio': 2295.0, 'amazon': 2295.0, 'amazon_fba': 2295.0, 'flipkart': 2295.0, 'limeroad': 2295.0, 'myntra': 2295.0, 'paytm': 2295.0, 'snapdeal': 2295.0}
  Products Available: 143
  Average Price: ₹None
  Price Range: ₹{'ajio': 2295.0, 'amazon': 2295.0, 'amazon_fba': 2295.0, 'flipkart': 2295.0, 'limeroad': 2295.0, 'myntra': 2295.0, 'paytm': 2295.0, 'snapdeal': 2295.0} - ₹{'ajio': 2295.0,

### 6.3 Catalog Analysis


In [63]:
# MongoDB Aggregation: Analyze by catalog
print("=== Aggregation: Catalog Analysis ===")
catalog_stats = mongo.aggregate_catalog_analysis()

print("\nCatalog Statistics:")
for stat in catalog_stats:
    print(f"\nCatalog: {stat.get('catalog')}")
    print(f"  Total Products: {stat.get('total_products')}")
    print(f"  Unique Categories: {stat.get('unique_categories')}")
    print(f"  Unique SKUs: {stat.get('unique_skus')}")
    print(f"  Average Price: ₹{stat.get('avg_price')}")

# SQL Equivalent:
# SELECT 
#     catalog,
#     COUNT(*) as total_products,
#     COUNT(DISTINCT category) as unique_categories,
#     COUNT(DISTINCT sku) as unique_skus,
#     AVG(avg_price) as avg_price
# FROM products
# GROUP BY catalog
# ORDER BY total_products DESC;


=== Aggregation: Catalog Analysis ===

Catalog Statistics:

Catalog: Colors-8
  Total Products: 192
  Unique Categories: 1
  Unique SKUs: 48
  Average Price: ₹1895.0

Catalog: Moments
  Total Products: 144
  Unique Categories: 1
  Unique SKUs: 36
  Average Price: ₹2294.92

Catalog: Breeze-4
  Total Products: 64
  Unique Categories: 1
  Unique SKUs: 16
  Average Price: ₹2195.0


### 6.4 Price Comparison Analysis


In [64]:
# MongoDB Aggregation: Find products with price variation
print("=== Aggregation: Products with Price Variation ===")
price_comparison = mongo.aggregate_price_comparison()

print(f"\nFound {len(price_comparison)} products with price variation")
print("\nTop 10 products with highest price variation:")
for i, product in enumerate(price_comparison[:10], 1):
    print(f"\n{i}. SKU: {product.get('sku')}")
    print(f"   Category: {product.get('category')}")
    print(f"   Price Variation: ₹{product.get('price_variation')}")
    print(f"   Platforms: {list(product.get('platforms', {}).keys())}")


=== Aggregation: Products with Price Variation ===

Found 1 products with price variation

Top 10 products with highest price variation:

1. SKU: Os206_3141_S
   Category: Kurta
   Price Variation: ₹95.0
   Platforms: ['ajio', 'amazon', 'amazon_fba', 'flipkart', 'limeroad', 'myntra', 'paytm', 'snapdeal']


## 7. MongoDB vs SQL Comparison

### Key Differences:

#### Flexibility
- **MongoDB**: Schema-less, can store nested documents, easy to add new fields
- **SQL**: Fixed schema, requires ALTER TABLE for schema changes

#### Data Structure
- **MongoDB**: Document-based (JSON-like), nested objects
- **SQL**: Table-based, normalized or denormalized

#### Query Language
- **MongoDB**: JavaScript-like query syntax, aggregation pipelines
- **SQL**: Declarative SQL queries

#### Example: Finding products by category

**MongoDB:**
```python
products = collection.find({"product.category": "Kurta"})
```

**SQL:**
```sql
SELECT * FROM products WHERE category = 'Kurta';
```

#### Example: Aggregation

**MongoDB Aggregation Pipeline:**
```python
pipeline = [
    {"$group": {
        "_id": "$product.category",
        "avg_price": {"$avg": "$metadata.avg_price"}
    }}
]
```

**SQL:**
```sql
SELECT category, AVG(avg_price) 
FROM products 
GROUP BY category;
```

### When to Use MongoDB:
- Semi-structured or unstructured data
- Rapid schema evolution
- Nested/hierarchical data
- Horizontal scaling needs
- Document-based data model fits your use case

### When to Use SQL:
- Structured, relational data
- Complex joins and transactions
- ACID compliance critical
- Mature ecosystem and tools
- Team familiar with SQL


## 8. Cleanup and Close Connection


In [65]:
# Close MongoDB connection
mongo.close()
print("Connection closed successfully")


MongoDB connection closed
Connection closed successfully
